# 计算需要的工人数以及对应工龄

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#几条生产线需要几个工人
def staff_number(lines):
    work_nums = 3
    work_days = 5
    return int(work_nums*lines*7/work_days)

opt_staff_df = pd.read_excel('./A题-全部数据/附件3/操作人员信息表.xlsx')
#计算需要的工人数量
staff_nums = staff_number(10)
print('一共需要{}个工人'.format(staff_nums))

#计算各个工龄的操作人员占所有操作人员里的百分比
rate = opt_staff_df.groupby('工龄').apply(lambda x: len(x) / len(opt_staff_df))
rate = pd.DataFrame(rate, columns=['rate'])
rate

一共需要42个工人


rate
工龄      
1    0.2
2    0.2
3    0.1
4    0.2
5    0.2
6    0.1

In [86]:
staffs = np.floor(rate['rate']*staff_nums+0.5)
staffs[5] += 1
staffs[4] += 1
# 使用reset_index()方法将Series转换为DataFrame，并为另一列命名为"数量"
staffs = staffs.reset_index()
staffs.columns = ['工龄', '数量']
staffs

工龄   数量
0   1  8.0
1   2  8.0
2   3  4.0
3   4  9.0
4   5  9.0
5   6  4.0

In [87]:
# 初始化开始编号
start = 1

# 为每位工人分配编号
def assign_ids(row):
    global start  # 使用全局变量start来更新开始编号
    ids = ["B{:03d}".format(i) for i in range(start, start + int(row['数量']))]
    start += int(row['数量'])  # 更新start为下一工龄组的开始编号
    return ids

# 创建一个新列，存储编号
staffs['IDs'] = staffs.apply(assign_ids, axis=1)

# 展开编号列表为单独的行
all_ids = []
for index, row in staffs.iterrows():
    for worker_id in row['IDs']:
        all_ids.append({'工龄': row['工龄'], 'ID': worker_id})

# 创建一个新的DataFrame
worker_df = pd.DataFrame(all_ids)

# 显示结果
worker_df = worker_df.iloc[:,[1,0]]
worker_df

ID  工龄
0   B001   1
1   B002   1
2   B003   1
3   B004   1
4   B005   1
5   B006   1
6   B007   1
7   B008   1
8   B009   2
9   B010   2
10  B011   2
11  B012   2
12  B013   2
13  B014   2
14  B015   2
15  B016   2
16  B017   3
17  B018   3
18  B019   3
19  B020   3
20  B021   4
21  B022   4
22  B023   4
23  B024   4
24  B025   4
25  B026   4
26  B027   4
27  B028   4
28  B029   4
29  B030   5
30  B031   5
31  B032   5
32  B033   5
33  B034   5
34  B035   5
35  B036   5
36  B037   5
37  B038   5
38  B039   6
39  B040   6
40  B041   6
41  B042   6

In [88]:
import pandas as pd
import numpy as np

# 假设有10条生产线和7天的排班需求
num_days = 7
num_lines = 10
shift_types = ['早', '中', '晚']

# 初始化工人ID列表
worker_ids = worker_df['ID'].tolist()
num_workers = len(worker_ids)

# 创建一个排班数组，其中包含每位工人每天的班次，初始设为'休'
schedule = np.full((num_workers, num_days), '休')

# 每天每个班次需要的工人数
workers_per_shift_per_day = num_lines

# 初始化每位工人的剩余工作天数
remaining_work_days = np.full(num_workers, 5)

# 确保每天每个班次都有10个工人
# 重构班次分配
for day in range(num_days):
    for shift in shift_types:
        # 找到还有剩余工作天数的工人
        available_workers = [i for i in range(num_workers) if remaining_work_days[i] > 0 and schedule[i, day] == '休']

        while True:
            if not available_workers:
                print(f"Warning: No available workers for day {day}, shift {shift}")
                break

            # 找到剩余工作天数的最大值
            max_work_days = max(remaining_work_days[worker] for worker in available_workers)
            # 选择剩余工作天数等于最大值的工人
            max_day_workers = [worker for worker in available_workers if remaining_work_days[worker] == max_work_days]

            if len(max_day_workers) >= workers_per_shift_per_day:
                selected_workers = np.random.choice(max_day_workers, workers_per_shift_per_day, replace=False)
                schedule[selected_workers, day] = shift
                remaining_work_days[selected_workers] -= 1
                break
            else:
                # 首先选取所有最大值工人
                if max_day_workers:
                    schedule[max_day_workers, day] = shift
                    remaining_work_days[max_day_workers] -= 1
                    remaining_needed = workers_per_shift_per_day - len(max_day_workers)
                
                # 减少最大工作天数并重新选择工人
                for worker in available_workers:
                    if remaining_work_days[worker] == max_work_days:
                        remaining_work_days[worker] -= 1
                
                # 重新确定新一轮的最大工作天数工人
                available_workers = [i for i in available_workers if remaining_work_days[i] > 0 and schedule[i, day] == '休']
                max_work_days = max(remaining_work_days[worker] for worker in available_workers if worker not in max_day_workers)
                max_day_workers = [worker for worker in available_workers if remaining_work_days[worker] == max_work_days]
                
                if len(max_day_workers) >= remaining_needed:
                    selected_workers = np.random.choice(max_day_workers, remaining_needed, replace=False)
                    schedule[selected_workers, day] = shift
                    remaining_work_days[selected_workers] -= 1
                    break
                elif not max_day_workers:
                    print(f"Critical warning: Cannot meet worker requirements for day {day}, shift {shift} even after adjustments.")
                    break

# 转换为DataFrame
df_schedule = pd.DataFrame(schedule, index=worker_ids, columns=np.arange(1, num_days + 1))

# 格式1的DataFrame
schedule_format1 = df_schedule.reset_index()
schedule_format1.rename(columns={'index': '日期'}, inplace=True)
schedule_format1 = schedule_format1.T
schedule_format1.columns = schedule_format1.iloc[0].values
schedule_format1 = schedule_format1.iloc[1:,:]


# 准备数据以构建格式2的DataFrame
schedule_data = []
for day in range(1, num_days + 1):
    for shift in shift_types:
        workers_in_shift = df_schedule[df_schedule[day] == shift].index
        row = {'日期': day, '班次': shift}
        row.update({f'M3{i+1:02d}': worker for i, worker in enumerate(workers_in_shift)})
        schedule_data.append(row)

# 格式2的DataFrame
schedule_format2 = pd.DataFrame(schedule_data)

# 打印结果
print("排班格式1:")
print(schedule_format1)
print("\n排班格式2:")
print(schedule_format2)


排班格式1:
  B001 B002 B003 B004 B005 B006 B007 B008 B009 B010  ... B033 B034 B035 B036  \
1    中    休    晚    早    休    早    中    休    中    休  ...    早    晚    中    早   
2    晚    早    休    晚    早    中    晚    早    休    早  ...    中    休    中    晚   
3    晚    中    中    晚    中    休    晚    晚    早    早  ...    晚    早    休    休   
4    休    早    休    休    晚    早    休    早    中    晚  ...    休    休    晚    中   
5    中    晚    早    晚    中    休    晚    晚    休    中  ...    中    早    休    中   
6    休    晚    早    晚    晚    早    休    晚    早    中  ...    晚    早    中    休   
7    早    休    晚    休    休    晚    晚    休    中    休  ...    休    中    中    中   

  B037 B038 B039 B040 B041 B042  
1    中    休    休    晚    早    晚  
2    休    早    早    中    休    晚  
3    晚    早    晚    休    早    休  
4    晚    休    早    晚    早    中  
5    休    早    中    晚    早    晚  
6    早    早    休    休    中    晚  
7    晚    晚    早    中    休    休  

[7 rows x 42 columns]

排班格式2:
    日期 班次  M301  M302  M303  M304  M305  M306  M3

In [89]:
schedule_format2

日期 班次  M301  M302  M303  M304  M305  M306  M307  M308  M309  M310
0    1  早  B004  B006  B012  B014  B015  B018  B032  B033  B036  B041
1    1  中  B001  B007  B009  B021  B022  B023  B025  B026  B035  B037
2    1  晚  B003  B013  B016  B017  B019  B024  B028  B034  B040  B042
3    2  早  B002  B005  B008  B010  B011  B027  B030  B031  B038  B039
4    2  中  B006  B018  B019  B020  B021  B028  B029  B033  B035  B040
5    2  晚  B001  B004  B007  B014  B015  B023  B026  B032  B036  B042
6    3  早  B009  B010  B016  B017  B027  B030  B031  B034  B038  B041
7    3  中  B002  B003  B005  B011  B012  B013  B020  B022  B024  B029
8    3  晚  B001  B004  B007  B008  B015  B025  B028  B033  B037  B039
9    4  早  B002  B006  B008  B013  B017  B021  B024  B025  B039  B041
10   4  中  B009  B011  B016  B022  B027  B029  B031  B032  B036  B042
11   4  晚  B005  B010  B012  B014  B018  B020  B030  B035  B037  B040
12   5  早  B003  B012  B013  B019  B023  B025  B026  B034  B038  B041
13   5  中  B001  B005  B010  B017  B027  B029  B031  B033  B036  B039
14   5  晚  B002  B004  B007  B008  B011  B016  B018  B024  B040  B042
15   6  早  B003  B006  B009  B015  B019  B021  B030  B034  B037  B038
16   6  中  B010  B014  B020  B022  B023  B026  B028  B032  B035  B041
17   6  晚  B002  B004  B005  B008  B013  B016  B017  B031  B033  B042
18   7  早  B001  B011  B014  B018  B023  B026  B027  B028  B029  B039
19   7  中  B009  B019  B020  B022  B024  B032  B034  B035  B036  B040
20   7  晚  B003  B006  B007  B012  B015  B021  B025  B030  B037  B038

In [90]:
schedule_format1.to_excel('./results4-1.xlsx')
schedule_format2.to_excel('./results4-2.xlsx',index= None)
worker_df.to_csv('./操作人员信息表.csv')